## Use stacked images (3D) and Efficientnet3D model

Use models with only one MRI type, then ensemble the 4 models 


In [ ]:
import os
import sys 
import glob
import random
import time

import numpy as np
import pandas as pd
import pydicom
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch import nn
from torch.utils import data as torch_data
from sklearn import model_selection as sk_model_selection
from torch.nn import functional as torch_functional
import torch.nn.functional as F

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

import imgaug as ia
import imgaug.augmenters as iaa

In [ ]:
dataset = 'test'
series_names = ['FLAIR','T1w','T1wCE','T2w']
directory = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'
# This function gets called by get_middle_images function
#Returns the list of all images present in a particular series(modality) of a given patient
def get_series_list(dataset, study_id, series_name):

    series_list = []

    for subdirs, dirs, files in os.walk(directory + '/' + dataset + '/' + study_id + "/" + series_name):
        series_list = os.listdir(directory + '/' + dataset + '/' + study_id + '/' + series_name)
    return series_list


def get_middle_images(study_id):
    
    middle_images = []
    
    # Iterate through each of the four series directories and get the files 
    for ser in series_names:
        series_files = get_series_list(dataset, study_id, ser)
        series_df = pd.DataFrame(columns = ['image','instance_number'])

        # Get the DICOM InstanceNumber tag to order the images since we can't rely on the filenames to be in order
        for s in series_files:
            img = pydicom.dcmread(directory + "/" + dataset + "/" + study_id + "/" + ser + "/" + s)
            series_df.loc[len(series_df.index)] = [s, img[0x0020,0x0013].value]
            
            # 0x0020,0x0013 refers to image number, comes from Dicom dictionary (https://imagej.nih.gov/nih-image/download/nih-image_spin-offs/NucMed_Image/DICOM%20Dictionary)
 
        series_df['instance_number'] = pd.to_numeric(series_df['instance_number'])

        # Sort the image list by InstanceNumber
        series_df = series_df.sort_values(by=['instance_number'])
        
        # Find the image in the middle of the list
        middle_index = int(series_df.shape[0] / 2)
        middle_image = series_df.iloc[middle_index]['image']

        middle_images.append(ser + "/" + middle_image)

    return middle_images


#Given the image orientation, returns the image plane 
def get_image_plane(loc):

    row_x = round(loc[0])
    row_y = round(loc[1])
    row_z = round(loc[2])
    col_x = round(loc[3])
    col_y = round(loc[4])
    col_z = round(loc[5])

    if row_x == 1 and row_y == 0 and col_x == 0 and col_y == 0:
        return "Coronal"

    if row_x == 0 and row_y == 1 and col_x == 0 and col_y == 0:
        return "Sagittal"

    return 'Axial'

#for getting the image plane corresponding to the middle images of all the series of a particular patient
def plot_images(images, image_id):
    result = []
    for img in images:
        image = pydicom.dcmread(directory + "/" + dataset + "/" + image_id + "/" + img)
        # 0x0020,0x0037 in dicom dictionary refers to "Image Orientation (Patient)"
        image_orientation_patient = image[0x0020,0x0037]
        plane = get_image_plane(image_orientation_patient)
        result.append(plane)
        
    return result

df_sub = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv")
# x:05 converts columns to have a fixed width(to include leading zeroes)
df_sub['BraTS21ID'] = df_sub['BraTS21ID'].apply(lambda x: f"{x:05}")
image_plane_mid_images = df_sub['BraTS21ID'].apply(lambda x: plot_images(get_middle_images(x), x))
print(image_plane_mid_images.head())
print('--------------------------------------')
data = pd.DataFrame.from_dict(dict(zip(image_plane_mid_images.index, image_plane_mid_images.values))).T
print(data.head())
print('--------------------------------------')

df_sub[['FLAIR', 'T1w', 'T1wCE', 'T2w']] = 0
df_sub[['FLAIR', 'T1w', 'T1wCE', 'T2w']] = data
print(df_sub.head())

In [ ]:
dataset = 'train'
series_names = ['FLAIR','T1w','T1wCE','T2w']
directory = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'
# This function gets called by get_middle_images function
#Returns the list of all images present in a particular series(modality) of a given patient
def get_series_list(dataset, study_id, series_name):

    series_list = []

    for subdirs, dirs, files in os.walk(directory + '/' + dataset + '/' + study_id + "/" + series_name):
        series_list = os.listdir(directory + '/' + dataset + '/' + study_id + '/' + series_name)
    return series_list


def get_middle_images(study_id):
    
    middle_images = []
    
    # Iterate through each of the four series directories and get the files 
    for ser in series_names:
        series_files = get_series_list(dataset, study_id, ser)
        series_df = pd.DataFrame(columns = ['image','instance_number'])

        # Get the DICOM InstanceNumber tag to order the images since we can't rely on the filenames to be in order
        for s in series_files:
            img = pydicom.dcmread(directory + "/" + dataset + "/" + study_id + "/" + ser + "/" + s)
            series_df.loc[len(series_df.index)] = [s, img[0x0020,0x0013].value]
            
            # 0x0020,0x0013 refers to image number, comes from Dicom dictionary (https://imagej.nih.gov/nih-image/download/nih-image_spin-offs/NucMed_Image/DICOM%20Dictionary)
 
        series_df['instance_number'] = pd.to_numeric(series_df['instance_number'])

        # Sort the image list by InstanceNumber
        series_df = series_df.sort_values(by=['instance_number'])
        
        # Find the image in the middle of the list
        middle_index = int(series_df.shape[0] / 2)
        middle_image = series_df.iloc[middle_index]['image']

        middle_images.append(ser + "/" + middle_image)

    return middle_images


#Given the image orientation, returns the image plane 
def get_image_plane(loc):

    row_x = round(loc[0])
    row_y = round(loc[1])
    row_z = round(loc[2])
    col_x = round(loc[3])
    col_y = round(loc[4])
    col_z = round(loc[5])

    if row_x == 1 and row_y == 0 and col_x == 0 and col_y == 0:
        return "Coronal"

    if row_x == 0 and row_y == 1 and col_x == 0 and col_y == 0:
        return "Sagittal"

    return 'Axial'

#for getting the image plane corresponding to the middle images of all the series of a particular patient
def plot_images(images, image_id):
    result = []
    for img in images:
        image = pydicom.dcmread(directory + "/" + dataset + "/" + image_id + "/" + img)
        # 0x0020,0x0037 in dicom dictionary refers to "Image Orientation (Patient)"
        image_orientation_patient = image[0x0020,0x0037]
        plane = get_image_plane(image_orientation_patient)
        result.append(plane)
        
    return result

df_train = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
df_train['BraTS21ID'] = df_train['BraTS21ID'].apply(lambda x: f"{x:05}")
image_plane_mid_images = df_train['BraTS21ID'].apply(lambda x: plot_images(get_middle_images(x), x))
data = pd.DataFrame.from_dict(dict(zip(image_plane_mid_images.index, image_plane_mid_images.values))).T
print(data.head())
df_train[['FLAIR', 'T1w', 'T1wCE', 'T2w']] = 0
df_train[['FLAIR', 'T1w', 'T1wCE', 'T2w']] = data
print(df_train.head())


In [ ]:
import pickle

df_train.to_pickle('train')
df_sub.to_pickle('sub')


In [ ]:
data_directory = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'
pytorch3dpath = "../input/efficientnetpyttorch3d/EfficientNet-PyTorch-3D"
    
mri_types = ['FLAIR','T1w','T1wCE','T2w']
angle_types = [ 'Sagittal', 'Axial', 'Coronal']
SIZE = 256
NUM_IMAGES = 64

sys.path.append(pytorch3dpath)
from efficientnet_pytorch_3d import EfficientNet3D

## Functions to load images

In [ ]:
from pydicom.pixel_data_handlers import apply_voi_lut
def load_dicom_image(path, img_size=SIZE):
    dicom = pydicom.read_file(path)
    #VOI LUT is used to transform raw DICOM data to "human-friendly" view
    #pixel_array returns a numpy.ndarray containing the pixel data

    data = apply_voi_lut(dicom.pixel_array, dicom)
    # depending on this value, X-ray may look inverted - fix that:
    if dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    data = cv2.resize(data, (img_size, img_size))
    return data

def load_dicom_images_3d(scan_id, num_imgs=NUM_IMAGES, img_size=SIZE, mri_type="FLAIR", split="train"):

    files = sorted(glob.glob(f"{data_directory}/{split}/{scan_id}/{mri_type}/*.dcm"))
    
    middle = len(files)//2
    num_imgs2 = num_imgs//2
    p1 = max(0, middle - num_imgs2)
    p2 = min(len(files), middle + num_imgs2)
    img3d = np.stack([load_dicom_image(f) for f in files[p1:p2]]).T 
    if img3d.shape[-1] < num_imgs:
        n_zero = np.zeros((img_size, img_size, num_imgs - img3d.shape[-1]))
        img3d = np.concatenate((img3d,  n_zero), axis = -1)
            
    return np.expand_dims(img3d,0)

a = load_dicom_images_3d("00000")
a.shape

In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

set_seed(42)

## train / test splits


In [ ]:
# train_df = pd.read_csv(f"{data_directory}/train_labels.csv")
display(df_train)

df_train, df_valid = sk_model_selection.train_test_split(
    df_train, 
    test_size=0.2, 
    random_state=10, 
    stratify=df_train["MGMT_value"],
)

## Model and training classes

In [ ]:

#Label smoothing is a regularization technique for classification problems to prevent the 
#model from predicting the labels too confidently during training and generalizing poorly

class Dataset(torch_data.Dataset):
    def __init__(self, paths, targets=None, mri_type=None, label_smoothing=0.01, split="train"):
        self.paths = paths
        self.targets = targets
        self.mri_type = mri_type
        self.label_smoothing = label_smoothing
        self.split = split
          
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):
        scan_id = self.paths[index]
        if self.targets is None:
            data = load_dicom_images_3d(str(scan_id).zfill(5), mri_type=self.mri_type[index], split=self.split)
        else:
            data = load_dicom_images_3d(str(scan_id).zfill(5), mri_type=self.mri_type[index], split="train")

        if self.targets is None:
            return {"X": torch.tensor(data).float(), "id": scan_id}
        else:
            y = torch.tensor(abs(self.targets[index]-self.label_smoothing), dtype=torch.float)
            return {"X": torch.tensor(data).float(), "y": y}


In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': 2}, in_channels=1)
        n_features = self.net._fc.in_features
        self.net._fc = nn.Linear(in_features=n_features, out_features=1, bias=True)
    
    def forward(self, x):
        out = self.net(x)
        return out

In [ ]:
class Trainer:
    def __init__(
        self, 
        model, 
        device, 
        optimizer, 
        criterion
    ):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.criterion = criterion

        self.best_valid_score = np.inf
        self.n_patience = 0
        self.lastmodel = None
        
    def fit(self, epochs, train_loader, valid_loader, save_path, patience):        
        for n_epoch in range(1, epochs + 1):
            self.info_message("EPOCH: {}", n_epoch)
            
            train_loss, train_time = self.train_epoch(train_loader)
            valid_loss, valid_auc, valid_time = self.valid_epoch(valid_loader)
            
            self.info_message(
                "[Epoch Train: {}] loss: {:.4f}, time: {:.2f} s            ",
                n_epoch, train_loss, train_time
            )
            
            self.info_message(
                "[Epoch Valid: {}] loss: {:.4f}, auc: {:.4f}, time: {:.2f} s",
                n_epoch, valid_loss, valid_auc, valid_time
            )

            # if True:
            # if self.best_valid_score < valid_auc: 
            if self.best_valid_score > valid_loss: 
                self.save_model(n_epoch, save_path, valid_loss, valid_auc)
                self.info_message(
                     "valid loss decreased from {:.4f} to {:.4f}. Saved model to '{}'", 
                    self.best_valid_score, valid_loss, self.lastmodel
                )
                self.best_valid_score = valid_loss
                self.n_patience = 0
            else:
                self.n_patience += 1
            
            if self.n_patience >= patience:
                self.info_message("\nValid loss didn't improve in last {} epochs.", patience)
                break
            
    def train_epoch(self, train_loader):
        self.model.train()
        t = time.time()
        sum_loss = 0

        for step, batch in enumerate(train_loader, 1):
            X = batch["X"].to(self.device)
            targets = batch["y"].to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(X).squeeze(1)
            
            loss = self.criterion(outputs, targets)
            loss.backward()

            sum_loss += loss.detach().item()

            self.optimizer.step()
            
            message = 'Train Step {}/{}, train_loss: {:.4f}'
            self.info_message(message, step, len(train_loader), sum_loss/step, end="\r")
        
        return sum_loss/len(train_loader), int(time.time() - t)
    
    def valid_epoch(self, valid_loader):
        self.model.eval()
        t = time.time()
        sum_loss = 0
        y_all = []
        outputs_all = []

        for step, batch in enumerate(valid_loader, 1):
            with torch.no_grad():
                X = batch["X"].to(self.device)
                targets = batch["y"].to(self.device)

                outputs = self.model(X).squeeze(1)
                loss = self.criterion(outputs, targets)

                sum_loss += loss.detach().item()
                y_all.extend(batch["y"].tolist())
                outputs_all.extend(outputs.tolist())

            message = 'Valid Step {}/{}, valid_loss: {:.4f}'
            self.info_message(message, step, len(valid_loader), sum_loss/step, end="\r")
            
        y_all = [1 if x > 0.5 else 0 for x in y_all]
        auc = roc_auc_score(y_all, outputs_all)
        
        return sum_loss/len(valid_loader), auc, int(time.time() - t)
    
    def save_model(self, n_epoch, save_path, loss, auc):
        self.lastmodel = f"{save_path}-e{n_epoch}-loss{loss:.3f}-auc{auc:.3f}.pth"
        torch.save(
            {
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.optimizer.state_dict(),
                "best_valid_score": self.best_valid_score,
                "n_epoch": n_epoch,
            },
            self.lastmodel,
        )
    
    @staticmethod
    def info_message(message, *args, end="\n"):
        print(message.format(*args), end=end)

## train models

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_mri_type(df_train, df_valid, mri_type, angle_type):

    df_train = df_train[df_train[mri_type]==angle_type]
    df_valid = df_valid[df_valid[mri_type]==angle_type]
    df_train.loc[:,"MRI_Type"] = mri_type
    df_valid.loc[:,"MRI_Type"] = mri_type

    print(df_train.shape, df_valid.shape)
    display(df_train.head())
    
    train_data_retriever = Dataset(
        df_train["BraTS21ID"].values, 
        df_train["MGMT_value"].values, 
        df_train["MRI_Type"].values,
    )

    valid_data_retriever = Dataset(
        df_valid["BraTS21ID"].values, 
        df_valid["MGMT_value"].values,
        df_valid["MRI_Type"].values
    )

    train_loader = torch_data.DataLoader(
        train_data_retriever,
        batch_size=4,
        shuffle=True,
        num_workers=8,
    )

    valid_loader = torch_data.DataLoader(
        valid_data_retriever, 
        batch_size=4,
        shuffle=False,
        num_workers=8,
    )

    model = Model()
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    criterion = torch_functional.binary_cross_entropy_with_logits

    trainer = Trainer(
        model, 
        device, 
        optimizer, 
        criterion
    )

    history = trainer.fit(
        50, 
        train_loader, 
        valid_loader, 
        f"{mri_type}-{angle_type}", 
        10,
    )
    
    return trainer.lastmodel

modelfiles = None

if not modelfiles:
    for m in mri_types:
        for a in angle_types:
            modelfiles = train_mri_type(df_train, df_valid, m,a)
print(modelfiles)

## Predict function

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
modeldict = {
    'FLAIR': {
        'Axial': './FLAIR-Axial-e5-loss0.684-auc0.574.pth',
        'Coronal': '../input/effnets3d/FLAIR-Coronal-e6-loss0.624-auc0.556.pth',
        'Sagittal': '../input/effnets3d/FLAIR-Sagittal-e1-loss0.694-auc0.425.pth',
    },
    
    'T1w': {
        'Axial': '../input/effnets3d/T1w-Axial-e16-loss0.680-auc0.595.pth',
        'Coronal': '../input/effnets3d/T1w-Coronal-e1-loss0.693-auc0.500.pth',
        'Sagittal': '../input/effnets3d/T1w-Sagittal-e1-loss0.696-auc0.500.pth',
    },
    
    'T1wCE': {
        'Axial': '../input/effnets3d/T1wCE-Axial-e3-loss0.693-auc0.623.pth',
        'Coronal': '../input/effnets3d/T1wCE-Coronal-e15-loss0.680-auc0.583.pth',
        'Sagittal': '../input/effnets3d/T1wCE-Sagittal-e13-loss0.671-auc0.375.pth',
    },
    
    'T2w': {
        'Axial': '../input/effnets3d/T2w-Axial-e10-loss0.686-auc0.604.pth',
        'Coronal': '../input/effnets3d/T2w-Coronal-e24-loss0.663-auc0.500.pth',
        'Sagittal': '../input/effnets3d/T2w-Sagittal-e1-loss0.693-auc0.475.pth',
    }
}

In [ ]:
def predict(modelfile, df, mri_type, angle_type, split):
    print("Predict:", modelfile, mri_type, angle_type, df.shape)
    
    df = df[df[mri_type]==angle_type]
    df.loc[:,"MRI_Type"] = mri_type
    data_retriever = Dataset(
        df.BraTS21ID.values, 
        mri_type=df["MRI_Type"].values,
        split=split
    )

    data_loader = torch_data.DataLoader(
        data_retriever,
        batch_size=4,
        shuffle=False,
        num_workers=2,
    )
   
    model = Model()
    model.to(device)
    
    checkpoint = torch.load(modelfile, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.eval()
    
    y_pred = []
    ids = []

    for e, batch in enumerate(data_loader,1):
        print(f"{e}/{len(data_loader)}", end="\r")
        with torch.no_grad():
            tmp_pred = torch.sigmoid(model(batch["X"].to(device))).cpu().numpy().squeeze()
            if tmp_pred.size == 1:
                y_pred.append(tmp_pred)
            else:
                y_pred.extend(tmp_pred.tolist())
            ids.extend(batch["id"])
            
    preddf = pd.DataFrame({"BraTS21ID": ids, "MGMT_value": y_pred}) 
    preddf = preddf.set_index("BraTS21ID")
    return preddf

## Ensemble for submission

In [ ]:
submission = df_sub.copy()
submission["MGMT_value"] = 0
for mtype in mri_types:
    for atype in angle_types:
        m = modeldict[mtype][atype]
        try:
            pred = predict(m, submission, mtype, atype, split="test")
        except ValueError:
            continue
        submission = pd.merge(submission, pred, how='left', on='BraTS21ID').fillna(0)
        submission['MGMT_value'] = submission['MGMT_value_x'] + submission['MGMT_value_y']
        submission = submission[['BraTS21ID', 'MGMT_value', 'FLAIR', 'T1w', 'T1wCE', 'T2w']]

submission = submission[['BraTS21ID', 'MGMT_value']]
submission["MGMT_value"] /= len(mri_types)

<!-- submission.to_csv("submission.csv", index=False) -->

In [ ]:
submission.to_csv("submission.csv", index=False)

In [ ]:
submission = pd.read_csv('./submission.csv')

In [ ]:
submission